# p10. 데이터 집계& 그룹연산

## 1. Groupby

### (1) Series를 data Groupby 하기

In [3]:

import numpy as np
import pandas as pd
df=pd.DataFrame({'key1':['a','a','b','b','a'],
                              'key2':["one","two","one","two","one"],
                            'data1': np.random.randn(5),
                            'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.691445,-0.281915
1,a,two,0.868442,0.014547
2,b,one,0.837937,0.605149
3,b,two,0.755657,0.131038
4,a,one,0.138443,0.881205


In [4]:
#key1열이 a인 행들에 대해서 data1의 평균구하기
grouped=df['data1'].groupby(df['key1'])  #분리(Split)
grouped

In [5]:
grouped.mean() # Apply(적용)-결합(Combine)

key1
a   -1.245850
b   -0.207353
Name: data1, dtype: float64

In [6]:
# 그룹화의 기준이 될 열을 2개이상 지정할 때
means=df["data1"].groupby([df['key1'], df["key2"]]).mean()
means

key1  key2
a     one    -0.950139
      two    -1.837270
b     one     0.048375
      two    -0.463080
Name: data1, dtype: float64

In [7]:
# unstack() : 최하위 인덱스(key2) => column
means.unstack()

key2,one,two
key1,,
a,-0.950139,-1.83727
b,0.048375,-0.46308


### (2) DataFrame을 Groupby 하기

In [8]:
df.groupby("key1").mean()

,data1,data2
key1,,
a,-1.245850,0.445489
b,-0.207353,-0.276478


In [9]:
#count(): 행의 갯수
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


In [10]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.950139  0.660665
     two  -1.837270  0.015137
b    one   0.048375 -1.832369
     two  -0.463080  1.279412

In [11]:
#특정 열에 대한 통계량 산출하기
df.groupby(['key1','key2'])['data2'].mean()

key1  key2
a     one     0.660665
      two     0.015137
b     one    -1.832369
      two     1.279412
Name: data2, dtype: float64

### (3) 반복문을 이용하여 그룹화 직후의 결과물 확인

In [12]:
df.groupby('key1') # 그룹화 실행후 결과물 확인 안됨=>반복문 사용

In [13]:
for name, group in df.groupby('key1'):  #반복문을 통해서 그룹화 결과물 출력
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -2.268023  0.801801
1    a  two -1.837270  0.015137
4    a  one  0.367744  0.519528
b
  key1 key2     data1     data2
2    b  one  0.048375 -1.832369
3    b  two -0.463080  1.279412


In [14]:
# 기준열이 2개인 경우, name변수 자리에 소괄호()를 
# 이용해서 2개변수에 각 기준열의 성분을 받는다.
for (k1,k2), group in df.groupby(['key1', 'key2']):  #반복문을 통해서 그룹화 결과물 출력
    print(k1,k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one -2.268023  0.801801
4    a  one  0.367744  0.519528
a two
  key1 key2    data1     data2
1    a  two -1.83727  0.015137
b one
  key1 key2     data1     data2
2    b  one  0.048375 -1.832369
b two
  key1 key2    data1     data2
3    b  two -0.46308  1.279412


### (4) 딕셔너리를 이용한 그룹화 직후의 결과물 보기


In [15]:
#dict의 그룹화는 list로 쪼개서 담은 다음 dictionary에 한꺼번에 담는다.
pieces=dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one -2.268023  0.801801
 1    a  two -1.837270  0.015137
 4    a  one  0.367744  0.519528, 'b':   key1 key2     data1     data2
 2    b  one  0.048375 -1.832369
 3    b  two -0.463080  1.279412}

In [16]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.048375,-1.832369
3,b,two,-0.463080,1.279412


### (5) 기준열 대신에 별도로 정의된 Series나 dict를 이용해서 Mapping & 데이터 그룹화

In [21]:
df2=pd.DataFrame(np.random.randn(5,5),
                columns=['a','b','c','d','e'],
                 index=['A','B','C','D','E']
                )
df2

,a,b,c,d,e
A,-0.066391,0.405276,-0.189657,0.516705,0.054757
B,1.712123,-1.262612,-2.244585,-0.789445,0.805026
C,-0.851913,-0.445103,2.039811,0.003607,-0.801059
D,-1.076373,0.750837,-0.102580,1.494489,-0.892907
E,1.765785,2.036114,0.112255,-0.164353,0.383727


In [22]:
map_dict={'a':'red', 'b':'red', 'c':'blue', 'd':'blue', 'e':'red', 'f':'orange'}
map_dict

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

In [20]:
dict(list(df2.groupby(map_dict, axis=1)))

{'blue':           c         d
 A  1.187919 -1.299628
 B -0.624267  0.369598
 C  1.379095  0.465088
 D -0.607337 -0.875829
 E -0.460876 -1.158778, 'red':           a         b         e
 A -0.231048 -1.022188  0.974302
 B  1.404679  1.970313 -1.140230
 C  0.464143 -0.245424 -1.926456
 D -0.641998 -0.733862  0.415366
 E -0.648013  0.758254 -2.557367}

In [23]:
# 합계구하기
df2.groupby(map_dict, axis=1).sum()

,blue,red
A,0.327048,0.393642
B,-3.034030,1.254538
C,2.043417,-2.098075
D,1.391909,-1.218443
E,-0.052097,4.185625


In [24]:
# 행의 수 구하기
map_s=pd.Series(map_dict)
df2.groupby(map_s, axis=1).count()

,blue,red
A,2,3
B,2,3
C,2,3
D,2,3
E,2,3


### (6) 데이터 그룹화에 사용자 정의함수 적용하기

In [4]:
grouped=df.groupby("key1")

def peak_to_peak(arr):
    return arr.max()-arr.min()
##agg(): 각 그룹마다 사용자 함수 적용
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.729999,1.163120
b,0.082280,0.474112


In [5]:
# 각 그룹들에서 표준편차 구하기
grouped.agg("std")

,data1,data2
key1,,
a,0.380797,0.604405
b,0.058181,0.335247


In [6]:
# 그룹화된 결과물에 전체 통계량
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.566110  0.380797  0.138443  0.414944  0.691445  0.779943   
b      2.0  0.796797  0.058181  0.755657  0.776227  0.796797  0.817367   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.868442   3.0  0.204613  0.604405 -0.281915 -0.133684  0.014547   
b     0.837937   2.0  0.368093  0.335247  0.131038  0.249565  0.368093   

                          
           75%       max  
key1                      
a     0.447876  0.881205  
b     0.486621  0.605149